# koGPT2 for Question Generation

In [1]:
# !python -m nltk.downloader punkt

/opt/conda/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# !git clone https://github.com/codertimo/KorQuAD-Question-Generation.git

Cloning into 'KorQuAD-Question-Generation'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 125 (delta 54), reused 105 (delta 39), pack-reused 0
Receiving objects: 100% (125/125), 926.44 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [3]:
# !mv KorQuAD-Question-Generation QG

In [4]:
%cd QG

/opt/ml/input/data/QG


In [9]:
import random
from argparse import ArgumentParser

import pandas as pd
import torch
from tokenizers import SentencePieceBPETokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2LMHeadModel

from korquad_qg.config import QGConfig
from korquad_qg.dataset import MAX_QUESTION_SPACE, MIN_QUESTION_SPACE, QGDataset, QGDecodingDataset, load_korquad_dataset

In [6]:
config = QGConfig()

In [32]:
MODEL_PATH = "/opt/ml/input/data/QG/QG_kogpt2.pth"

In [38]:
model = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2")
model.load_state_dict(torch.load('/opt/ml/input/data/QG/QG_kogpt2.pth', map_location="cpu"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
tokenizer = SentencePieceBPETokenizer.from_file(
    vocab_filename="tokenizer/vocab.json", merges_filename="tokenizer/merges.txt", add_prefix_space=False
)

## 1. 데이터 준비

In [109]:
import pandas as pd
from ast import literal_eval

ke_data = pd.read_csv('/opt/ml/input/data/KEdata_for_generation_v2.csv')
ke_data['answers'] = ke_data['answers'].map(literal_eval)

In [110]:
len(ke_data)

3883

In [44]:
def split_answers(dataset):
    answer_start, text = [], []

    for i in range(len(dataset)):
        df = dataset.iloc[i]
        answer_start.append(df['answers']['answer_start'][0])
        answer = df['answers']['text'][0]
        text.append(answer)


    new_dataset = pd.DataFrame({'title': dataset['title'], 
                            'context' : dataset['context'], 
                            'question': dataset['question'], 
                            'id': dataset['id'],
                            'answer': text,
                            'answer_start': answer_start, 
                            'document_id': dataset['document_id'], 
                            '__index_level_0__': dataset['__index_level_0__'],})
    
    return new_dataset

In [111]:
ke_data = split_answers(ke_data)
ke_data.head(3)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,NaN,mrc-1-003954,공무원,268,18293,3954
1,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,NaN,mrc-1-003955,웅촌면,62,47334,3955
2,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,NaN,mrc-1-003956,울주군,58,47334,3956


In [22]:
from typing import NamedTuple, Optional
class QAExample(NamedTuple):
    context: str
    answer: str
    question: Optional[str] = None

In [112]:
examples_list = []
cnt=0
for idx in range(len(ke_data)):
    text = ke_data["context"][idx]
    title = ke_data["answer"][idx]
    document_id = idx

    example = QAExample(text, title)
    tmp = [example,document_id]
    examples_list.append(tmp)

In [114]:
examples=[]
d_id=[]
for i in examples_list:
    examples.append(i[0])
    d_id.append(i[1])

In [115]:
examples[:3]

[QAExample(context='미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05', answer='공무원', question=None),
 QAExample(context='동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기의 유물로, 경상남도 울주군

In [116]:
dataset = QGDecodingDataset(examples, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
model = model.to(device)
model.eval()

## 2. Generate

In [119]:
generated_results = []

for i, batch in tqdm(enumerate(dataloader), desc="generate", total=len(dataloader)):
    input_ids, attention_mask = (v.to(device) for v in batch)
    origin_seq_len = input_ids.size(-1)

    decoded_sequences = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=origin_seq_len + MAX_QUESTION_SPACE,
        min_length=origin_seq_len + MIN_QUESTION_SPACE,
        pad_token_id=0,
        bos_token_id=1,
        eos_token_id=2,
        num_beams=5,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
    )

    for decoded_tokens in decoded_sequences.tolist():
        decoded_question_text = tokenizer.decode(decoded_tokens[origin_seq_len:])
        decoded_question_text = decoded_question_text.split("</s>")[0].replace("<s>", "")
        generated_results.append(
            (i, examples[i].answer, examples[i].question, decoded_question_text, d_id[i])
        )

generate: 100%|██████████| 3883/3883 [46:00<00:00,  1.41it/s]


In [120]:
generated_df = pd.DataFrame(generated_results)
generated_df.columns = ['idx', 'answer', 'question', 'text', 'd_id']
generated_df.to_csv('generation_data_v2.csv', index=False)

In [121]:
len(generated_df)

3883

In [54]:
generated_df.describe(include='all')

,idx,answer,question,text,d_id
count,23718.000000,23718,0,23718,23718.000000
unique,NaN,9842,0,18133,NaN
top,NaN,상태,NaN,나카모리 아키나는 누구인가?,NaN
freq,NaN,176,NaN,15,NaN
mean,11858.500000,NaN,NaN,NaN,11858.500000
std,6846.941178,NaN,NaN,NaN,6846.941178
min,0.000000,NaN,NaN,NaN,0.000000
25%,5929.250000,NaN,NaN,NaN,5929.250000
50%,11858.500000,NaN,NaN,NaN,11858.500000
75%,17787.750000,NaN,NaN,NaN,17787.750000


In [122]:
generated_df[generated_df['text'].duplicated()==True]

,idx,answer,question,text,d_id
2,2,울주군,None,계사명 사리가 출토된 곳은?,2
7,7,조선말,None,고려 공민왕의 모후산은 어디인가?,7
9,9,전라남도,None,고려 공민왕의 모후산은 어디인가?,9
24,24,라파엘로,None,이탈리아 르네상스의 중요한 인물은?,24
48,48,이조좌랑,None,이괄이 즉위한 후 명문의 자제로 경연시독관이 된 사람은 누구인가?,48
...,...,...,...,...,...
3878,3878,작곡가,None,오토 니콜라이의 지휘자는 누구인가?,3878
3879,3879,드라고넷,None,고대 그리스에서 악마를 뜻하는 단어는?,3879
3880,3880,게오르기우스,None,그리스어에서 악마를 뜻하는 단어는 무엇인가?,3880
3881,3881,주도하,None,"기민련, 녹색당 연정은 무엇의 연정인가?",3881


## 3. Train data 합치기

In [62]:
%cd ..

/opt/ml/input/data


In [73]:
from datasets import load_dataset, load_from_disk

# dataset = load_dataset("csv", data_files={"train":"../data/train_dataset_random_aug.csv", "validation" : "../data/validation_dataset.csv"})
dataset = load_from_disk("../data/train_dataset/")

train_dataset = pd.DataFrame(dataset['train'])
val_dataset = pd.DataFrame(dataset['validation'])

/opt/conda/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [123]:
train_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [126]:
new_answers = []
for idx in range(len(ke_data)):
    new_answers.append({'answer_start': [ke_data.answer_start[idx]], 'text': [ke_data.answer[idx]]})

In [127]:
new_dataset = pd.DataFrame({'title': ke_data['title'], 
                            'context' : ke_data['context'], 
                            'question': generated_df['text'], 
                            'id': ke_data['id'],
                            'answers': new_answers,
                            'document_id': ke_data['document_id'], 
                            '__index_level_0__': ke_data['__index_level_0__'],})

In [128]:
new_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,상원의원은 무엇을 파면할 권한이 있는가?,mrc-1-003954,"{'answer_start': [268], 'text': ['공무원']}",18293,3954
1,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리가 출토된 곳은?,mrc-1-003955,"{'answer_start': [62], 'text': ['웅촌면']}",47334,3955
2,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리가 출토된 곳은?,mrc-1-003956,"{'answer_start': [58], 'text': ['울주군']}",47334,3956
3,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,1950년대 필리핀에서 사망한 큰아들의 이름은 무엇인가?,mrc-1-003957,"{'answer_start': [64], 'text': ['가가야']}",10426,3957
4,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,오싱이 결혼한 사람은 누구인가?,mrc-1-003958,"{'answer_start': [540], 'text': ['아가씨']}",10426,3958


In [130]:
total_dataset = pd.concat([train_dataset, new_dataset])

In [86]:
total_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [131]:
print(f"generated dataset count : {len(new_dataset)}")
print(f"train dataset count : {len(train_dataset)}")
print(f"TOTAL dataset count : {len(total_dataset)}")

generated dataset count : 3883
train dataset count : 3952
TOTAL dataset count : 7835


In [132]:
total_dataset.to_csv('QGaugmentation_dataset_v2.csv', index=False)